In [1]:
from utils.handle_data import get_integer_columns_and_dtype, get_cards_metadata, process_and_save_in_batches
import pandas as pd
import numpy as np
import random

In [2]:
d_type = get_integer_columns_and_dtype('Handled_data/draft_data_best_players.csv')
raw_data = pd.read_csv('Handled_data/draft_data_best_players.csv', dtype=d_type)

In [3]:
#groupby draft_id
draft_id_group = raw_data.groupby('draft_id')

In [4]:
#remove all group with less than 42 rows
draft_id_group = draft_id_group.filter(lambda x: len(x) == 42)

In [5]:
draft_id_group

,expansion,event_type,draft_id,draft_time,rank,event_match_wins,event_match_losses,pack_number,pick_number,pick,...,pool_Wose Pathfinder,pool_You Cannot Pass!,pool_Éomer of the Riddermark,"pool_Éomer, Marshal of Rohan","pool_Éowyn, Fearless Knight","pool_Éowyn, Lady of Rohan",pack_card_Dawn of a New Age,pool_Dawn of a New Age,user_n_games_bucket,user_game_win_rate_bucket
0,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,0,Ranger's Firebrand,...,0,0,0,0,0,0,0,0,50,0.52
1,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,1,Gollum's Bite,...,0,0,0,0,0,0,0,0,50,0.52
2,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,2,Erebor Flamesmith,...,0,0,0,0,0,0,0,0,50,0.52
3,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,3,Erebor Flamesmith,...,0,0,0,0,0,0,0,0,50,0.52
4,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,4,Hithlain Knots,...,0,0,0,0,0,0,0,0,50,0.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4491913,LTR,PremierDraft,44c333335108419f8bd1bdfcfb771d82,2023-08-28 02:47:49,mythic,2,3,2,9,Shelob's Ambush,...,1,1,0,0,0,0,0,0,100,0.52
4491914,LTR,PremierDraft,44c333335108419f8bd1bdfcfb771d82,2023-08-28 02:47:49,mythic,2,3,2,10,Took Reaper,...,1,1,0,0,0,0,0,0,100,0.52
4491915,LTR,PremierDraft,44c333335108419f8bd1bdfcfb771d82,2023-08-28 02:47:49,mythic,2,3,2,11,Stalwarts of Osgiliath,...,1,1,0,0,0,0,0,0,100,0.52
4491916,LTR,PremierDraft,44c333335108419f8bd1bdfcfb771d82,2023-08-28 02:47:49,mythic,2,3,2,12,Warbeast of Gorgoroth,...,1,1,0,0,0,0,0,0,100,0.52


In [6]:
# Seed for reproducibility
np.random.seed(42)

# Get a list of unique group identifiers (i.e., unique 'draft_id's)
unique_ids = draft_id_group['draft_id'].unique()

# Sample 20% of the unique identifiers for the test set
test_ids = np.random.choice(unique_ids, size=int(len(unique_ids) * 0.20), replace=False)

# Create a new list for the remaining ids (those not in test set)
remaining_ids = [id for id in unique_ids if id not in test_ids]

# Sample 25% of the remaining identifiers for the validation set (0.25 * 0.8 = 0.2)
val_ids = np.random.choice(remaining_ids, size=int(len(remaining_ids) * 0.25), replace=False)

# The rest (60% of total) goes to the train set
train_ids = [id for id in remaining_ids if id not in val_ids]

In [7]:
# Create the sets based on the sampled identifiers
test_set = draft_id_group[draft_id_group['draft_id'].isin(test_ids)]
val_set = draft_id_group[draft_id_group['draft_id'].isin(val_ids)]
train_set = draft_id_group[draft_id_group['draft_id'].isin(train_ids)]


#save data
test_set.to_csv('dataset/test_set.csv', index=False)
val_set.to_csv('dataset/val_set.csv', index=False)
train_set.to_csv('dataset/train_set.csv', index=False)

### Add metadata

In [8]:
#Load all cards data
meta_data_card = pd.read_csv('Handled_data/ltr_cards_data.csv')

In [9]:
meta_data_card.head(5)

,name,rarity,color_identity,cmc,colors
0,"Andúril, Flame of the West",mythic,T,3.0,T
1,"Aragorn and Arwen, Wed",mythic,M,6.0,M
2,"Aragorn, Company Leader",rare,M,3.0,M
3,"Aragorn, the Uniter",mythic,M,4.0,M
4,"Arwen, Mortal Queen",mythic,M,3.0,M


In [10]:
meta_data_card = get_cards_metadata('Handled_data/ltr_cards_data.csv')

In [11]:
meta_data_card.head(5)

,name,cmc,id,rarity_common,rarity_mythic,rarity_rare,rarity_uncommon,colors_B,colors_G,colors_M,colors_R,colors_T,colors_U,colors_W
0,"Andúril, Flame of the West",0.002318,0,0,1,0,0,0,0,0,0,1,0,0
1,"Aragorn and Arwen, Wed",1.977281,1,0,1,0,0,0,0,1,0,0,0,0
2,"Aragorn, Company Leader",0.002318,2,0,0,1,0,0,0,1,0,0,0,0
3,"Aragorn, the Uniter",0.660639,3,0,1,0,0,0,0,1,0,0,0,0
4,"Arwen, Mortal Queen",0.002318,4,0,1,0,0,0,0,1,0,0,0,0


In [12]:
process_and_save_in_batches('dataset/train_set.csv', meta_data_card, 'dataset/train',batch_size=42000)
process_and_save_in_batches('dataset/test_set.csv', meta_data_card, 'dataset/test',batch_size=42000)
process_and_save_in_batches('dataset/val_set.csv', meta_data_card, 'dataset/val',batch_size=42000)


In [ ]:
import pickle
with open("dataset/test_X_2.pkl", "rb") as f:
            batch_data = pickle.load(f)